

---



---

---


**Перед запуском кода загрузите файлы, из которых собираем БД, формат файлов .txt**





---





In [ ]:
### Установка фреймворков
!pip install -U langchain # общий фреймворк
!pip install langchain_community # для работы с загрузчиком документов
!pip install -U sentence-transformers # для
!pip install -U langchain-chroma # для создание БД
!pip install -U deep-translator # переводчик (здесь используем Google Translator)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67

ВНИМАНИЕ! БЛОК КОДА НИЖЕ ЭТО ПРОСТО ФУНКЦИИ И ТЕСТ РАБОТОСПОСОБНОСТИ ПЕРЕВОДЧИК

In [ ]:



"""Переводчик вопроса (запроса)"""
from deep_translator import GoogleTranslator

def query_translation(query) -> str:

  translated_query = GoogleTranslator(source='ru', target='en').translate(query)

  return translated_query


"""Переводчик ответа от LLM (генератора)"""
def response_translation(response) -> str:

  translated_response = GoogleTranslator(source='en', target='ru').translate(response)

  return translated_response


"""Переводчик файлов для Базы знаний"""
# (используется разово для импорта новых файлов в Базу знаний)
def text_translation(path_new_knowledge_file) -> str:

  translated_file = GoogleTranslator(source='ru', target='en').translate_file(path_new_knowledge_file)

  return translated_file # Вместо translated_file сделать импорт переведенных файлов в Базу Знаний


"""Тест работоспособности переводчика"""
# Пример запроса на русском языке
query = "Как работает система RAG?"

# Обработка запроса с переводом
response = query_translation(query)

# Вывод результата
print(response)






How does the RAG system work?


In [ ]:
# Загрузчик документов
from langchain.document_loaders import DirectoryLoader, TextLoader

# в loader указываем путь в папку с базой знаний
# path - укажите путь к ПАПКЕ с файлами для Базы знаний ()
# glob - файлы какого формата будут учитываться, тут ".txt" формат
# с другими форматами не тестил

loader = DirectoryLoader(path='/content', glob = './*.txt', loader_cls= TextLoader)
documents = loader.load()


#  Ссылка на сайт для подробного понимания DirectoryLoader (зажми 'Ctrl' и нажми на ссылку)
#  https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.directory.DirectoryLoader.html

In [ ]:
# Разбинеие на чанки
from langchain.text_splitter import RecursiveCharacterTextSplitter

# chunk_size - регулирует размер чанков
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200)
documents = text_splitter.split_documents(documents)

In [ ]:
# Эмбеддинг
from langchain.embeddings import HuggingFaceBgeEmbeddings

# Берём модель для эмбединга с Hugging Face
model_name = 'BAAI/bge-small-en-v1.5'
encode_kwargs = {'normalize_embeddings': True}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    # вместо 'cpu' можно указать 'cuda' для более быстрой работы
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Создание Базы знаний через Chroma (без сохранения Базы знаний)
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(documents, bge_embeddings)


"""
Добавить загрузку документов формата .txt
и парсинг информативных сайтов для обновления Базы Знаний(обновление скорее всего будет в настоящем времени)
Под обновлением подразумевается доступ к сайтам по URL ссылки
"""



'\nДобавить загрузку документов формата .txt\nи парсинг информативных сайтов для обновления Базы Знаний(обновление скорее всего будет в настоящем времени)\nПод обновлением подразумевается доступ к сайтам по URL ссылки\n'

In [ ]:
#  Ретривер
# as_retriever извлекает из векторной БД чанки
retriever = vectordb.as_retriever()

# response имеет вид списка, у него буквально тип данных 'list'

query = query_translation('Что такое венчурство') # Перевод вопроса с русского на англ

response_not_translated = retriever.invoke(query) # Запрос найденных чанков из БД


# используем response[0], потому что response - список,
#  а к списку не может быть применён метод page_content
# метод page_content выводит ответ в человечном виде
# о да, обязательно попробуй запустить без page_content)

response = response_translation(response_not_translated[0].page_content) # Перевод ответа модели

print(response)

инвесторы? -- Стартапы должны брать примерно столько денег, сколько им нужно на год, независимо от их дохода. Такой подход делает инвесторов более лояльными, поскольку он согласуется с их финансовыми


In [ ]:
# И да доп. инфу и подробности можно искать через документацию langchain
# мужики, там есть поиск в правой верхнем углу (на сайте langchain)
# Вот ссылка (зажми 'Ctrl' и нажми на ссылку)
# https://python.langchain.com/docs/introduction/
